In [1]:
import os
import scenic

# ### NuScenes Query
# from scenic.simulators.carla.nusc_query_api import NuscQueryAPI
# nusc = NuscQueryAPI(version='v1.0-trainval', \
#                     dataroot='/Users/edwardkim/Desktop/Scenic_Query/nuscenes_data')

/Users/edwardkim/Desktop/Scenic_Query/Scenic/src/scenic/core/errors.py:160: UserWarning: unable to install sys.excepthook to format Scenic backtraces
  warnings.warn('unable to install sys.excepthook to format Scenic backtraces')


In [2]:
### Dependency Analysis
def cacheExprTreeNodes(attribute, nodeSet=None):
    """cache all the nodes of the input attribute's expression tree to the dictionary"""
    if nodeSet is None:
        nodeSet = set()
    nodeSet.add(attribute)
    if attribute._dependencies == ():
        return nodeSet
    for dep in attribute._dependencies:
        cacheExprTreeNodes(dep, nodeSet)
    return nodeSet

def cacheAttributes(scenario, attributeList):
    dictionary = {}
    dictionary['objAttributes_names'] = []
    dictionary['positionAttributes_names'] = []
    dictionary['headingAttributes_names'] = []
    
    # cache all object attributes
    for i in range(len(scenario.original_objects)):
        obj = scenario.original_objects[i]
        obj_name = 'obj'+str(i)
        dictionary[obj_name] = {}
        
        for attribute in attributeList:
            dictionary[obj_name][attribute] = {}
            dictionary[obj_name][attribute]['self'] = getattr(obj, attribute)
            dictionary[obj_name][attribute]['set'] = cacheExprTreeNodes(getattr(obj, attribute), None)
            dictionary[obj_name][attribute]['intermediate_variables_set'] = []
            dictionary[obj_name][attribute]['dependent_attribute_names'] = []
            dictionary[obj_name][attribute]['jointly_dependent_attribute_names'] = []
            dictionary[obj_name][attribute]['dependent_attributes_objs'] = set()
            dictionary[obj_name][attribute]['jointly_dependent_attributes_objs'] = set()
            dictionary['objAttributes_names'].append(obj_name+"_"+attribute)
            if attribute == 'position':
                dictionary['positionAttributes_names'].append(obj_name+"_"+attribute)
            if attribute == 'heading':
                dictionary['headingAttributes_names'].append(obj_name+"_"+attribute)
                
    return dictionary

def checkDependenceOnAnotherAttribute(intersection, attr1_name, attr2_name, dictionary):
    """ checks whether the two attr1 and attr2 are jointly dependent on an intermediate variable
    or is both dependent on another attribute. 
    Output:
    True, if attr1 and attr2 are "dependent" on another attribute, not intermediate variable
    False, attr1 and attr2 are both "jointly dependent" on an intermediate variable
    """
    [obj1_name, attr1] = attr1_name.split('_')
    attr1_obj = dictionary[obj1_name][attr1]['self']
    attr1_jointly_dep_attr_names = dictionary[obj1_name][attr1]['jointly_dependent_attribute_names']
    [obj2_name, attr2] = attr2_name.split('_')
    attr2_obj = dictionary[obj2_name][attr2]['self']
    attr2_jointly_dep_attr_names = dictionary[obj2_name][attr2]['jointly_dependent_attribute_names']
#     print("checkDependenceOnAnotherAttribute attr1_name: ", attr1_name)
#     print("checkDependenceOnAnotherAttribute attr2_name: ", attr2_name)
    original_intersection = intersection
    
    objAttributes_names = dictionary['objAttributes_names'] 
    for attr_name in objAttributes_names:
        if attr_name == attr1_name:
            continue
        elif attr_name == attr2_name:
            break
        else:
            [obj_name, attr] = attr_name.split('_')
            attr_obj = dictionary[obj_name][attr]['self']
            attr_depSet = dictionary[obj_name][attr]['dependent_attribute_names']
            
            if attr_obj in original_intersection and attr_name not in attr1_jointly_dep_attr_names \
                and attr_name not in attr2_jointly_dep_attr_names: 
#                 print("other attr_name in the intersection: ", attr_name)
                attr_cachedSet = dictionary[obj_name][attr]['set']
                original_intersection = original_intersection - attr_cachedSet
#                 print("len(original_intersection): ", len(original_intersection))
                if len(original_intersection) == 0:
#                     print("returns True")
                    # the intersection is another attribute
                    return True
    return False
        
def findAttribute(other_attr_obj, attr_dict, dictionary):
    for obj_attr in attr_dict['dependent_attribute_names']:
        [obj, attr] = obj_attr.split("_")
        if other_attr_obj is dictionary[obj][attr]['self']:
            return obj_attr
    return None

def checkIntermediateSetMembership(attr_obj, attrIntermediateList):
    for intermediateSet in attrIntermediateList:
        if attr_obj in intermediateSet:
            return True
    return False

def analysis(objAttributes_names, dictionary):
    for i in range(len(objAttributes_names)):
        for j in range(len(objAttributes_names)):
            if i < j:
                attr1_name = objAttributes_names[i]
                attr2_name = objAttributes_names[j]
                [obj_name1, attr1] = attr1_name.split('_')
                [obj_name2, attr2] = attr2_name.split('_')
        
                attribute1 = dictionary[obj_name1][attr1]
                attribute2 = dictionary[obj_name2][attr2]
                attr1_obj = attribute1['self']
                attr2_obj = attribute2['self']
                
                set1 = attribute1['set']
                set2 = attribute2['set']
                intersection = set1.intersection(set2)
                
                if attr1_obj in intersection and attr1_obj not in attribute2['dependent_attributes_objs']:
                    # attr2_obj is dependent on attr1_obj
                    attribute2['dependent_attribute_names'].append(attr1_name)
                    attribute2['dependent_attributes_objs'].add(attr1_obj)
                elif attr2_obj in intersection and attr2_obj not in attribute1['dependent_attributes_objs']:
                    # jointly_dependent case (e.g. depedendencyAnalysisTest4.scenic)
                    if attr2_name not in attribute1['jointly_dependent_attribute_names']:
                        attribute1['jointly_dependent_attribute_names'].append(attr2_name)
                        attribute1['jointly_dependent_attributes_objs'].add(attr2_obj)     
                        attribute1['intermediate_variables_set'].append(intersection)

                    if attr1_name not in attribute2['jointly_dependent_attribute_names']:
                        attribute2['jointly_dependent_attribute_names'].append(attr1_name)
                        attribute2['jointly_dependent_attributes_objs'].add(attr1_obj)
                        attribute2['intermediate_variables_set'].append(intersection)
                        
                elif len(intersection) > 0 \
                    and attr1_obj not in intersection and attr2_obj not in intersection \
                    and not checkDependenceOnAnotherAttribute(intersection, attr1_name, attr2_name, dictionary):
                    # the two attributes are jointly dependent (i.e. share intermediate variable(s))
                    if attr2_name not in attribute1['jointly_dependent_attribute_names']:
                        attribute1['jointly_dependent_attribute_names'].append(attr2_name)
                        attribute1['jointly_dependent_attributes_objs'].add(attr2_obj)     
                        attribute1['intermediate_variables_set'].append(intersection)
                    
                    if attr1_name not in attribute2['jointly_dependent_attribute_names']:
                        attribute2['jointly_dependent_attribute_names'].append(attr1_name)
                        attribute2['jointly_dependent_attributes_objs'].add(attr1_obj)
                        attribute2['intermediate_variables_set'].append(intersection)

                else:
                    pass
    return dictionary
    
def dependencyAnalysis(scenario, attributeList):
    dictionary = cacheAttributes(scenario, attributeList)
    dictionary['numberOfObjects'] = len(scenario.original_objects)
    objAttributes_names = dictionary['objAttributes_names']
    dictionary = analysis(objAttributes_names, dictionary)
    return dictionary

def sortDependency(dictionary, scenario, monolithic_translation=False):
    output = []
    covered_attributes = []
    
    if not monolithic_translation:
        for elem in dictionary['objAttributes_names']:
            if elem in covered_attributes:
                continue
            covered_attributes.append(elem)
            [obj_name, attr_name] = elem.split("_")
            joint_dep_set = dictionary[obj_name][attr_name]['jointly_dependent_attribute_names']
            if len(joint_dep_set) > 0:
                jointly_dependent_list = [elem]

                for j in joint_dep_set:
                    [j_obj_name, j_attr] = j.split("_")
                    jointly_dependent_list.append(j)
                    covered_attributes.append(j)
                output.append(jointly_dependent_list)
            else:
                output.append([elem])
    else:
        output = [(dictionary['objAttributes_names'])]
    return output

In [3]:
# ## Test Dependency Analysis

# attributeList = ['position', 'heading']
# d = dependencyAnalysis(scenario, attributeList)
# print(sortDependency(d, scenario))

In [19]:
### SMT Translation Pipeline

from scenic.core.regions import SectorRegion
from scenic.core.vectors import OrientedVector, Vector
from scenic.core.distributions import *
from scenic.domains.driving.roads import Network
from scenic.core.regions import PointInRegionDistribution
from scenic.core.type_support import TypecheckedDistribution
import subprocess
import itertools
import numpy as np
import os

def resetConditionedVar(obj):
    obj._conditioned = obj
    if (obj._dependencies is None):
        return None
    for dep in obj._dependencies:
        resetConditionedVar(dep)
    return None

def unconditionAllAttributes(scenario):
    for obj in scenario.objects:
        resetConditionedVar(obj.position)
        resetConditionedVar(obj.heading)
        
def extractLabelAttribute(label, obj_index, attribute_name, objType, dataType, correspondence, egoObjIndex):
    # Extract specific attribute from a label generated from a scenic program
    output = None
    if obj_index != egoObjIndex:
        if obj_index > egoObjIndex:
            obj_index -= 1
#         print("extractLabelAttribute()")
#         print("label[objType]: ", label[objType])
#         print("correspondence: ", correspondence)
#         print("correspondence[obj_index]: ", correspondence[obj_index])
#         print("attribute_name: ", attribute_name)
        output = label[objType][correspondence[obj_index]][attribute_name]
        if attribute_name == 'position':
            return Vector(output[0], output[1])
    else:
        output = label['EgoCar'][attribute_name]
        if attribute_name == 'position':
            return Vector(output[0], output[1])
    assert(output is not None)
    if dataType == 'nuScenes':
        output = math.radians(output+90) #90 deg added to reorient to Scenic's global coordinate system
    return output
        
def initializeSMTFile(smt_file_path):
    if os.path.isfile(smt_file_path):
        os.remove(smt_file_path)
    
    open(smt_file_path, 'w').close()
    writeSMTtoFile(smt_file_path, '(set-logic QF_NRA)')
    
def resetDictionary(cached_variables, smt_file_path):
    regionAroundEgo = cached_variables['regionAroundEgo']
    cached_variables.clear()
    cached_variables['variables'] = []
    cached_variables['regionAroundEgo'] = regionAroundEgo
    cached_variables['regionAroundEgo_polygon'] = regionAroundEgo.polygon
    cached_variables['smt_file_path'] = smt_file_path

def translateAttributeExpressionTree(attribute_name, attr_obj, attr_label, cached_variables, \
                                   dictionary, errorBound, debug=False):
    
    x_error_margin = str(errorBound['x'])
    y_error_margin = str(errorBound['y'])
    heading_error_margin = str(errorBound['heading'])
    
    ## TODO: add error bound range to attributes
    smt_file_path = cached_variables['smt_file_path']
    obj_name, attr_type = attribute_name.split("_")

    # Encode the given attribute's expression tree
    smt_var = attr_obj.encodeToSMT(smt_file_path, cached_variables, debug = debug)
    
    if attr_type == 'position':
        assert(isinstance(attr_label, Vector))
        x, y = smt_var
        (x_label, y_label) = (str(attr_label.x), str(attr_label.y))
        x_cond1 = smt_lessThanEq(smt_subtract(x_label, x_error_margin), x)
        x_cond2 = smt_lessThanEq(x, smt_add(x_label, x_error_margin))
        x_cond = smt_and(x_cond1, x_cond2)
        
        y_cond1 = smt_lessThanEq(smt_subtract(y_label, y_error_margin), y)
        y_cond2 = smt_lessThanEq(y, smt_add(y_label, y_error_margin))
        y_cond = smt_and(y_cond1, y_cond2)
        
#         (x_cond, y_cond) = vector_operation_smt((x_label, y_label), "equal",smt_var)
        writeSMTtoFile(smt_file_path, smt_assert(None, smt_and(x_cond, y_cond)))
    else:
        heading_label = str(attr_label)
        heading_cond1 = smt_lessThanEq(smt_subtract(heading_label, heading_error_margin), smt_var)
        heading_cond2 = smt_lessThanEq(smt_var, smt_add(heading_label, heading_error_margin))
        writeSMTtoFile(smt_file_path, smt_assert(None, smt_and(heading_cond1, heading_cond2)))
#         writeSMTtoFile(smt_file_path, smt_assert(None, smt_equal(str(attr_label), smt_var)))
        
def findObjType(obj):
    if "Car" in str(obj) or "Truck" in str(obj) or "Motorcycle" in str(obj) or "Bicycle" in str(obj):
        return "Vehicles"
    elif "Pedestrian" in str(obj):
        return "Pedestrians"
    elif "Cone" in str(obj):
        return "Objects"
    else:
        raise NotImplementedError
    return None

def conditionAttributes(jointlyDependentAttributeList, dictionary, scenario, dataType, correspondence, \
                        egoObjIndex, label):
    for attribute_name in jointlyDependentAttributeList:
        obj_name, attr_name = attribute_name.split("_")
        obj_index = int(obj_name.split("obj")[1])
        objType = findObjType(scenario.original_objects[obj_index])
        attr_label = extractLabelAttribute(label, obj_index, attr_name, objType, dataType, correspondence, \
                                              egoObjIndex)
        attr_obj = dictionary[obj_name][attr_name]['self']
        print("conditionAttributes attribute: ", attribute_name)
        print("conditionAttributes attr_label: ", attr_label)
        if isinstance(attr_label, float) or isinstance(attr_label, int):
            attr_obj.conditionTo(Constant(attr_label))
        elif isinstance(attr_obj, PointInRegionDistribution):
            attr_obj.conditionTo(attr_label)
            if isinstance(attr_obj.region, TypecheckedDistribution): 
                attr_obj.region.dist.conditionTo(attr_label)
            else:
                attr_obj.region.conditionTo(attr_label)
        else:
            attr_obj.conditionTo(attr_label)

def validateLabelElement(scenario, label, cached_variables, jointlyDependentAttributeList, dictionary, \
                         correspondence, egoObjIndex, dataType, errorBound, debug=False, falseTesting=False,\
                        monolithic_translation=False):
    
    count = 0
    ## translate jointly dependent attribute expression trees
    for attribute_name in jointlyDependentAttributeList:
        count += 1
        if count >= 2:
            break
        obj_name, attr_name = attribute_name.split("_")
        obj_index = int(obj_name.split("obj")[1])
        objType = findObjType(scenario.original_objects[obj_index])
        attr_label = extractLabelAttribute(label, obj_index, attr_name, objType, dataType, correspondence,\
                                          egoObjIndex)
        attr_obj = dictionary[obj_name][attr_name]['self']

        translateAttributeExpressionTree(attribute_name, attr_obj, attr_label, cached_variables, \
                                          dictionary, errorBound, debug)
        print("validateLabelElement encoding done for validateLabelElement: ", attribute_name)
#         if monolithic_translation:
#             print("validLabelElement: Monolithic translation case -- condition attribute: ",attribute_name)
#             conditionAttributes([attribute_name], dictionary, scenario, dataType, correspondence, \
#                         egoObjIndex, label)
    
    smt_file_path = cached_variables['smt_file_path']
    writeSMTtoFile(smt_file_path, "(check-sat)")
    writeSMTtoFile(smt_file_path, "(exit)")

    if subprocess.call("./run_smt_encoding.sh") == 1:
        return True
        
    return False

def generateObjectMatchingCorrespondenceSet(scenario):
    objTypeOrder = []
    objTypeDict = {}
    
    for obj in scenario.original_objects:
        if obj is not scenario.egoObject:
            objType = findObjType(obj)
            objTypeOrder.append(objType)

            if objType not in objTypeDict.keys():
                objTypeDict[objType] = {}
                objTypeDict[objType]['count'] = 1
            else:
                objTypeDict[objType]['count'] += 1
    
#     print("objTypeOrder: ", objTypeOrder)
    
    total_permutation_number = 1
    for objType in objTypeDict.keys():
        count = objTypeDict[objType]['count']
        index_list = [i for i in range(count)]
        objTypeDict[objType]['correspondence'] = list(itertools.permutations(index_list))
        total_permutation_number *= len(objTypeDict[objType]['correspondence'])
    
#     print("total_permutation_number: ", total_permutation_number)
#     print("objTypeDict['Vehicles']['count']: ", objTypeDict['Vehicles']['count'])
#     print("objTypeDict['Vehicles']['correspondence']: ", objTypeDict['Vehicles']['correspondence'])
#     print("len(objTypeDict['Vehicles']['correspondence']): ", len(objTypeDict['Vehicles']['correspondence']))
#     print("objTypeDict['Pedestrians']['count']", objTypeDict['Pedestrians']['count'])
#     print("objTypeDict['Pedestrians']['correspondence']", objTypeDict['Pedestrians']['correspondence'])
    
    # sort the types by the number of counts
    types = list(objTypeDict.keys())
    counts = [objTypeDict[objType]['count'] for objType in types]
    sorted_types = []
    sorted_types_nums = []
    
    for i in range(len(types)):
        elem = max(counts)
        index = counts.index(max(counts))
        sorted_types.append(types[index])
        sorted_types_nums.append(elem)
        del types[index]
        del counts[index]
    
#     print("sorted_types: ", sorted_types)
#     print("sorted_types_nums: ", sorted_types_nums)
    
    # compute the number of identical elements to insert per objType
    num_identicals = []
    for i in range(len(sorted_types)):
        if i == len(sorted_types)-1:
            num_identicals.append(1)
        else:
            num_identicals.append(np.prod(sorted_types_nums[i+1:]))
#     print("num_identicals: ", num_identicals)
    
    # create combinations of correspondences in the order of objTypeOrder
    correspondenceList = [[0]*len(objTypeOrder) for i in range(total_permutation_number)]
#     print("correspondenceList: ", correspondenceList)
    for i in range(len(sorted_types) - 1):
        objType = sorted_types[i]
        index = 0
#         print("objType: ", objType)
        
        for j in range(len(objTypeDict[objType]['correspondence'])):
            correspondenceToEdit = correspondenceList[index]
#             print("correspondenceToEdit: ", correspondenceToEdit)
            objTypeCorrespondence = objTypeDict[objType]['correspondence'][j]
#             print("objTypeCorrespondence: ", objTypeCorrespondence)
            correspondence = createCorrespondence(correspondenceToEdit, objType, objTypeOrder, \
                                                  objTypeCorrespondence)
#             print("correspondence: ", correspondence)
            
            for k in range(num_identicals[i]):
                correspondenceList[index] = correspondence
                index += 1
    
    finalCorrespondenceList = []
    if len(sorted_types) > 1:
        num_lastObjType_correspondence = len(objTypeDict[sorted_types[-1]]['correspondence'])
#         print("num_lastObjType_correspondence: ", num_lastObjType_correspondence)
        num_iteration = int(total_permutation_number / num_lastObjType_correspondence)
#         print("num_iteration: ", num_iteration)
        assert(total_permutation_number == num_iteration * num_lastObjType_correspondence)
        lastObjType = sorted_types[-1]
#         print("lastObjType: ", lastObjType)
        index = 0
        for m in range(num_iteration):
            for n in range(num_lastObjType_correspondence):
#                 print("index: ", index)
                correspondenceToEdit = correspondenceList[index]
#                 print("correspondenceToEdit: ", correspondenceToEdit)
                objTypeCorrespondence = objTypeDict[lastObjType]['correspondence'][n]
#                 print("objTypeCorrespondence: ", objTypeCorrespondence)
                correspondence = createCorrespondence(correspondenceToEdit, lastObjType, objTypeOrder, objTypeCorrespondence)
#                 print("correspondence: ", correspondence)
                finalCorrespondenceList.append(tuple(correspondence))
                index += 1
    else:
        finalCorrespondenceList = correspondenceList
    
    if finalCorrespondenceList == []: #case when there is only ego vehicle, no other obj
        finalCorrespondenceList = [(0)]
    return finalCorrespondenceList

def createCorrespondence(correspondence, objType, objTypeOrder, objTypeCorrespondence):
    index = 0
    for i in range(len(objTypeOrder)):
        if objTypeOrder[i] == objType:
            correspondence[i] = objTypeCorrespondence[index]
            index += 1
            if index == len(objTypeCorrespondence):
                break
    return correspondence
    

def findEgoObjIndex(scenario):
    for i in range(len(scenario.original_objects)):
        if scenario.original_objects[i] is scenario.egoObject:
            return i
    return None

def validateLabel(scenario, label, map_path, map_source='carla', ego_visibleDistance = 50, ego_viewAngle = 360, \
                  smt_file_path='./test_smt_encoding.smt2', attributeList = ['position', 'heading'], \
                  debug = False):
    #TODO: need to add object matching
    
    # Uncondition previously conditioned dependency objects
    unconditionAllAttributes(scenario)
    
    # Initialize smt file, if exists
    initializeSMTFile(smt_file_path)
    
    # Create Ego's VisibleRegion
    cached_variables = {}
#     if map_source == 'carla':
#         cached_variables['network'] = Network.fromFile(map_path, None)
#     elif map_source == 'nuScenes':
#         raise NotImplementedError
#     else:
#          raise NotImplementedError
    cached_variables['smt_file_path'] = smt_file_path
    cached_variables['variables'] = []
    label_ego_pos = label.egoObject.position
    label_ego_heading = label.egoObject.heading
    regionAroundEgo = SectorRegion(label_ego_pos, ego_visibleDistance, label_ego_heading, \
                                    math.radians(ego_viewAngle))
    cached_variables['regionAroundEgo'] = regionAroundEgo
    cached_variables['regionAroundEgo_polygon'] = regionAroundEgo.polygon
    dictionary = dependencyAnalysis(scenario, attributeList)
    sortedDependencyList = sortDependency(dictionary, scenario)
    print("sortedDependencyList: ", sortedDependencyList)
    
    for jointlyDependentAttributeList in sortedDependencyList:
        print(".........................validating : ", str(jointlyDependentAttributeList)+".......................")
        if not validateLabelElement(scenario, cached_variables, jointlyDependentAttributeList, dictionary, debug):
            print("NON-VALID ATTRIBUTES: ", jointlyDependentAttributeList)
            return False
        else: # condition attributes in jointlyDependentAttributeList
            print(".........................valid attribute: ", str(jointlyDependentAttributeList)+".......................")
            conditionAttributes(jointlyDependentAttributeList, dictionary, scenario, dataType, correspondence,\
                                   label)
            resetDictionary(cached_variables, regionAroundEgo, smt_file_path)
    
    ## Check Hard Constraint Satisfaction
    if not scenario.checkRequirements():
        return False
    
    return True

In [20]:
# import scenic
# scenic_script = "./examples/carla/ICCV_Scenic_Experiments/6_agent_scenario.scenic"
# scenario = scenic.scenarioFromFile(scenic_script)
# x = generateObjectMatchingCorrespondenceSet(scenario)
# print("finalOutput: ", x)
# print(len(x))
# # print(len(set(x)))

In [21]:
### Object Matching
import math

def conditionAllAttributes(scenario, dictionary, label, attributeList, correspondence, egoObjIndex, dataType):
    for obj_index in range(len(scenario.original_objects)):
        obj = scenario.original_objects[obj_index]
        for attribute_name in attributeList:
            objType = findObjType(obj)
            attr_label = extractLabelAttribute(label, obj_index, attribute_name, objType, \
                                               dataType, correspondence, egoObjIndex)
            if isinstance(attr_label, (float, int)):
                attr_label = Constant(attr_label)
            obj_attr = getattr(obj, attribute_name)
            obj_attr.conditionTo(attr_label)

def satisfyHardConstraints(scenario, dictionary, label, attributeList, correspondence, egoObjIndex, dataType):
    unconditionAllAttributes(scenario)
    conditionAllAttributes(scenario, dictionary, label, attributeList, correspondence, egoObjIndex, dataType)
    return scenario.checkRequirements()

def scenarioObjClassCount(scenario):
    # check whether the number of objects match per class
    objClassCountDict = {}
    for obj in scenario.original_objects:
        objType = findObjType(obj)
        if obj is not scenario.egoObject:
            if objType not in objClassCountDict.keys():
                objClassCountDict[objType] = {}
                objClassCountDict[objType]['count'] = 1
            else:
                objClassCountDict[objType]['count'] += 1
        else:
            objClassCountDict['EgoCar'] = {}
            objClassCountDict['EgoCar']['count'] = 1
    return objClassCountDict

def checkLabelValidity(label, objClassCountDict):
    for objType in objClassCountDict.keys():
        if objType == 'EgoCar':
            continue
        if len(label[objType]) != objClassCountDict[objType]['count']:
            print("label[objType]: ", label[objType])
            print("objClassCountDict[objType]['count']")
            return False
    return True

def queryLabelSetup(scenario, label, ego_visibleDistance = 50, ego_viewAngle = 360, \
                  smt_file_path='./test_smt_encoding.smt2', attributeList = ['position', 'heading'],\
                   dataType='carla', monolithic_translation=False):
    # Uncondition previously conditioned dependency objects
    unconditionAllAttributes(scenario)
    
    # Create Ego's VisibleRegion
    cached_variables = {}
    cached_variables['smt_file_path'] = smt_file_path
    cached_variables['variables'] = []
    (ego_x, ego_y) = label['EgoCar']['position']
    label_ego_pos = Vector(ego_x, ego_y)
    if dataType == 'carla':
        label_ego_heading = label['EgoCar']['heading']
    if dataType == 'nuScenes':
        label_ego_heading = math.radians(label['EgoCar']['heading']+90)
    regionAroundEgo = SectorRegion(label_ego_pos, ego_visibleDistance, label_ego_heading, \
                                    math.radians(ego_viewAngle))
    cached_variables['regionAroundEgo'] = regionAroundEgo
    cached_variables['regionAroundEgo_polygon'] = regionAroundEgo.polygon
    egoObjIndex = findEgoObjIndex(scenario)
    objClassCountDict = scenarioObjClassCount(scenario)
    
    # Sort Attribute Dependency 
    dictionary = dependencyAnalysis(scenario, attributeList)
    sortedDependencyList = sortDependency(dictionary, scenario, monolithic_translation)
    
    # Compute All Correspondence
    allObjCorrespondence = generateObjectMatchingCorrespondenceSet(scenario)
    
    outputDict = {}
    outputDict['cached_variables'] = cached_variables
    outputDict['sortedDependencyList'] = sortedDependencyList
    outputDict['allObjCorrespondence'] = allObjCorrespondence
    outputDict['egoObjIndex'] = egoObjIndex
    outputDict['dictionary'] = dictionary
    outputDict['objClassCountDict'] = objClassCountDict
    outputDict['attributeList'] = attributeList
    return outputDict
    
def queryLabel(scenario, label, outputDict, errorBound, dataType='carla', smt_file_path='./test_smt_encoding.smt2', \
               debug=False, monolithic_translation = False):
    
    objClassCountDict = outputDict['objClassCountDict']
    print("objClassCountDict: ", objClassCountDict)
    if not checkLabelValidity(label, objClassCountDict):
        # number of objects do not match per class ==> reject the label
        print("Obj Count does not match: Reject")
        return False
    
    cached_variables = outputDict['cached_variables']
    sortedDependencyList = outputDict['sortedDependencyList']
    allObjCorrespondence = outputDict['allObjCorrespondence']
    print("allObjCorrespondence: ", allObjCorrespondence)
    egoObjIndex = outputDict['egoObjIndex']
    dictionary = outputDict['dictionary']
    attributeList = outputDict['attributeList']
    
    # Uncondition previously conditioned dependency objects
    unconditionAllAttributes(scenario)
    
    # Initialize smt file, if exists
    initializeSMTFile(smt_file_path)
    
    print("begin query")
    for correspondence in allObjCorrespondence:
        failed = False
        print("queryLabel correspondence: ", correspondence)
        for jointlyDependentAttributeList in sortedDependencyList:
            print(".........................validating : ", str(jointlyDependentAttributeList)+".......................")
            if validateLabelElement(scenario, label, cached_variables, jointlyDependentAttributeList, dictionary, \
                                            correspondence, egoObjIndex, dataType, errorBound, debug=debug, \
                                            monolithic_translation=monolithic_translation):
                print(".........................valid attribute: ", str(jointlyDependentAttributeList)+".......................")
                conditionAttributes(jointlyDependentAttributeList, dictionary, scenario, dataType, \
                                    correspondence, egoObjIndex, label)
                resetDictionary(cached_variables, smt_file_path)
            else: # condition attributes in jointlyDependentAttributeList
                print("NON-VALID ATTRIBUTES: ", jointlyDependentAttributeList)
                failed = True
                unconditionAllAttributes(scenario)
                resetDictionary(cached_variables, smt_file_path)
                break

        ## Check Hard Constraint Satisfaction
        if not failed and satisfyHardConstraints(scenario, dictionary, label, attributeList, \
                                                 correspondence, egoObjIndex, dataType):
            return True
        if not failed:
            print("hard constraint not satisfied")

    return False

def convertScenicLabel(scenic_label):
    label = {}
    label['EgoCar'] = {}
    ego_pos = scenic_label.egoObject.position
    label['EgoCar']['position'] = (ego_pos[0], ego_pos[1])
    label['EgoCar']['heading'] = scenic_label.egoObject.heading
    label['Vehicles'] = []
    label['Pedestrians'] = []
    label['Objects'] = []
    for obj in scenic_label.objects:
        if obj is not scenic_label.egoObject:
            objType = findObjType(obj)
            objDict = {}
            objPos = obj.position
            objDict['position'] = (objPos[0], objPos[1])
            objDict['heading'] = obj.heading
            label[objType].append(objDict)
    return label

In [22]:
# import os
# import scenic
# scenic_script = "./examples/carla/ICCV_Human_Experiments/experiment1.scenic"
# scenario = scenic.scenarioFromFile(scenic_script)

# directory = '/Users/edwardkim/Desktop/Scenic_Query/nuscenes_data/experiment_results'
# subject1 = 'experiment_result_JayShenoy'
# subject2 = 'experiment_result_TaeSung'
# subject3 = 'experiment_result_Xiangyu'
# scenario_list = ['scenario1','scenario2','scenario3','scenario4','scenario5']
# dir3 = os.path.join(directory, 'scenario1_queried')
# # dir3 = os.path.join(directory, subject3, 'scenario3')
# filenames = [file for file in os.listdir(dir3) if file.endswith('.jpg')]
# img_name = filenames[0]
# label = nusc.get_img_data(img_name)
# ego_label = label['EgoCar']
# vehicles_label = label['Vehicles']
# pedestrians_label = label['Pedestrians']
# # print(ego_label)
# # print(vehicles_label)
# # print(pedestrians_label)
# count = 0
# # for file in filenames:
# file = 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535728830362404.jpg'
# count += 1
# label = nusc.get_img_data(file)
# outputDict = queryLabelSetup(scenario, label, ego_visibleDistance = 50, ego_viewAngle = 360, \
#                   smt_file_path='./test_smt_encoding.smt2', attributeList = ['position', 'heading'])

# if not queryLabel(scenario, label, outputDict, dataType='nuScenes', debug=True):
#     print("NON-VALID file: ", file)
# #     break
# else:
#     print("Valid "+str(count)+": ", file)


In [24]:
import scenic
scenic_script = "./examples/carla/ICCV_Scenic_Experiments/2_agent_scenario.scenic"
scenario = scenic.scenarioFromFile(scenic_script)
unconditionAllAttributes(scenario)
scenic_label, _ = scenario.generateForQuery(maxIterations=4000)
label = convertScenicLabel(scenic_label)
monolithic_translation = False
outputDict = queryLabelSetup(scenario, label, ego_visibleDistance = 30, ego_viewAngle = 360, \
                  smt_file_path='./test_smt_encoding.smt2', attributeList = ['position', 'heading'],\
                    dataType = 'carla', monolithic_translation=monolithic_translation)
print("label generated: ", len(scenic_label.objects))
print("converted label: ", label)
print(outputDict['sortedDependencyList'])
errorBound = {}
errorBound['x'] = 0.25 # meters == radius of the error margin ball around x
errorBound['y'] = 0.25 # meters == radius of the error margin ball around x
errorBound['heading'] = 0.0872 # radians = 5 degrees
if not queryLabel(scenario, label, outputDict, errorBound, dataType='carla', debug=False, monolithic_translation=monolithic_translation):
    print("NOT VALID")
else:
    print("Valid ")
    

/Users/edwardkim/Desktop/Scenic_Query/Scenic/src/scenic/simulators/carla/model.scenic:56: UserWarning: the "carla" package is not installed; will not be able to run dynamic simulations
  warnings.warn('the "carla" package is not installed; '


label generated:  2
converted label:  {'EgoCar': {'position': (-123.54152888724414, 36.415417224546985), 'heading': 0.12032092184061388}, 'Vehicles': [{'position': (-126.7286477647468, 43.02887203185274), 'heading': -2.692517952027882}], 'Pedestrians': [], 'Objects': []}
[['obj0_position', 'obj1_position'], ['obj0_heading', 'obj1_position'], ['obj1_heading']]
objClassCountDict:  {'EgoCar': {'count': 1}, 'Vehicles': {'count': 1}}
allObjCorrespondence:  [[0]]
begin query
queryLabel correspondence:  [0]
.........................validating :  ['obj0_position', 'obj1_position'].......................
validateLabelElement encoding done for validateLabelElement:  obj0_position
NON-VALID ATTRIBUTES:  ['obj0_position', 'obj1_position']
NOT VALID


In [9]:
'EgoCar': {'position': (195.0530128715603, 5.542774057505424), 'heading': -3.1250440581028167},x = scenario.original_objects[0].heading
print(type(x))
print(x)

SyntaxError: invalid syntax (<ipython-input-9-8a0533fc8e5b>, line 1)

In [ ]:
3.114852821762582 - math.pi

In [ ]:
# import os
# import scenic
# scenic_script = "./examples/carla/ICCV_Human_Experiments/experiment1.scenic"
# scenario = scenic.scenarioFromFile(scenic_script)
# # map_path = '/Users/edwardkim/Desktop/Scenic_Query/Scenic/tests/formats/opendrive/maps/CARLA/Town05.xodr'

# for i in range(1):
#     unconditionAllAttributes(scenario)
#     sample = scenario.generateForQuery(maxIterations = 4000, verbosity=0)
#     label, _ = sample
#     if not validateLabel(scenario, label, dataType='nuScenes', debug=False):
#         print("NOT VALID LABEL")
#         break
#     else:
#         print("label is valid: ", i)


In [ ]:
"""
Issue1: ahead/behind, left/right of uses the same heading angle as the referenced
        (1) As a result, position & heading are jointly dependent
        ==> what if we do not allow joint dependency between position and heading?
        This assumes that we can decouple joint dependency between the two, if exists.
        Is this true? Yes
        ==> Limitation: if many there are many jointly dependent features all at once, it may not be feasible to solve
        
        (2) an obj can have its position be dependent on its heading because its heading is the same as the 
        heading of another object to which the obj is depedent
        ==> is this only an issue with ego? because the ordering of the objects 
        ==> ==> solution: just keep the original objects ordering

Issue2: my assumption that jointly dependent and dependent relationships are disjoint is wrong
        (e.g. dependencyAnalysisTest4.scenic)
        ==> it's not possible to capture such case since the attribute contains the intermediate variable
        ==> another ordering process needs to be done within jointly dependent features based on dependence relations

Issue3: Need to check the case when multiple attributes are dependent on another attributes
        (e.g. )
        

Sorting Approach
Since the objects are listed in the order the scenario is written, 
the order in which SMT translation is to be done stays intact
The only issue now is to determine joint dependency
==> before adding to joint dependency, check whether the jointly dependent attribute is dependent on any of the
other jointly dependent attributes
"""